In [ ]:
import fiona
import geopandas as gpd

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
kmlfile = gpd.read_file("2018 Central Park Squirrel Census - Squirrel Data.kml")
print(kmlfile)


In [ ]:
kmlfile.crs

In [ ]:
import matplotlib.pyplot as plt
kmlfile.plot()
plt.title('Squirrels in Central Park (WGS84)')


In [ ]:
kmlfile.columns

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse("2018 Central Park Squirrel Census - Squirrel Data.kml")
root = tree.getroot()

# loop through the xml to parse it data point by data point
df = []
for datapoint_i in range(1,3023):
    elementdata = root[0][1][datapoint_i][1]
    df_row = []
    
    for x in elementdata:
        df_row.append(x[0].text)
    
      df.append(df_row)  

# get the column names
column_names = [x.attrib['name'] for x in elementdata]

# make into a dataframe and print
data = pd.DataFrame(df, columns = column_names)
data


In [ ]:
AM_data = data[data['shift'] == 'AM']
PM_data = data[data['shift'] == 'PM']


In [ ]:
AM_geodata = gpd.GeoDataFrame(AM_data, geometry=gpd.points_from_xy(AM_data['x'], AM_data['y']))
PM_geodata = gpd.GeoDataFrame(PM_data, geometry=gpd.points_from_xy(PM_data['x'], PM_data['y']))


In [ ]:
AM_geodata.plot()
plt.title('AM squirrels')

PM_geodata.plot()
plt.title('PM squirrels')


In [ ]:
import pandas as pd
flights_data = pd.read_csv('flights.csv')
flights_data


In [ ]:
geolookup = pd.read_csv('airports.csv')
geolookup


In [ ]:
# convert coordinates of geo lookup to a Point
# make to and from data set for to and from join
from shapely.geometry import Point
from_geo_lookup = geolookup[['IATA_CODE', 'LATITUDE', 'LONGITUDE']]
from_geo_lookup['geometry_from']= [Point(x,y) for x,y in zip( from_geo_lookup['LONGITUDE'], from_geo_lookup['LATITUDE'])]
from_geo_lookup = from_geo_lookup[['IATA_CODE','geometry_from' ]]

to_geo_lookup = geolookup[['IATA_CODE', 'LATITUDE', 'LONGITUDE']]
to_geo_lookup['geometry_to']= [Point(x,y) for x,y in zip( to_geo_lookup['LONGITUDE'], to_geo_lookup['LATITUDE'])]
to_geo_lookup = to_geo_lookup[['IATA_CODE','geometry_to' ]]


In [ ]:
# align data types for the joins
from_geo_lookup['IATA_CODE'] = from_geo_lookup['IATA_CODE'].map(str)
to_geo_lookup['IATA_CODE'] = to_geo_lookup['IATA_CODE'].map(str)

flights_data['ORIGIN_AIRPORT'] = flights_data['ORIGIN_AIRPORT'].map(str)
flights_data['DESTINATION_AIRPORT'] = flights_data['DESTINATION_AIRPORT'].map(str)
flights_data = flights_data[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']]


In [ ]:
flights_data = flights_data.merge(from_geo_lookup, left_on = 'ORIGIN_AIRPORT', right_on = 'IATA_CODE')
flights_data = flights_data.merge(to_geo_lookup, left_on = 'DESTINATION_AIRPORT', right_on = 'IATA_CODE')
flights_data = flights_data[['geometry_from', 'geometry_to']]
flights_data


In [ ]:
# convert points to lines
from shapely.geometry import LineString

lines = []
for i,row in flights_data.iterrows():
    try:
        point_from = row['geometry_from']
        point_to = row['geometry_to']
        lines.append(LineString([point_from, point_to]))
    except:
        pass
    
geodf = gpd.GeoDataFrame(lines, columns=['geometry'])
geodf


In [ ]:
# plot the lines
import matplotlib.pyplot as plt
geodf.plot(figsize=(12,12))
plt.title('the world as airpline trajectory lines')


In [ ]:
import geopandas as gpd
geojsonfile = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
print(geojsonfile)


In [ ]:
geojsonfile.plot()

In [ ]:
geojsonfile['area'] = geojsonfile['geometry'].apply(lambda x: x.area)
geojsonfile.sort_values('area').head(10)


In [ ]:
geojsonfile['length'] = geojsonfile['geometry'].apply(lambda x: x.length)
geojsonfile.sort_values('length', ascending=False).head(10)


In [ ]:
import rasterio
griddata = r'ore-kbz-mnt-litto3d-5m.tif'
img = rasterio.open(griddata)
matrix = img.read()
matrix


In [ ]:
from rasterio.plot import show
show(img)
